***12.1 # 6***


In [390]:
import pandas as pd


In [ ]:
from scipy.stats import f_oneway

# -- origin calculation
def findSST(df: pd.DataFrame):
    sum = 0
    grandMean = findGrandMean(df)
    for columnName in df.columns:
        column = df[columnName]
        sum = column.size*(column.mean() - grandMean)**2 + sum
    return sum

# -- origin calculation 
def findTSS (df: pd.DataFrame):
    sum = 0
    grandMean = findGrandMean(df)
    for columnName in df.columns:
        for element in df[columnName]:
            sum = (element - grandMean)**2 + sum
    return sum

# -- origin calculation
def findMSE (df: pd.DataFrame):
    numeratorSum = 0
    denominator = (df.size - df.shape[1])
    for columnName in df.columns:
        columnMean = df[columnName].mean()
        for element in df[columnName]:
            numeratorSum = ((element - columnMean) ** 2) + numeratorSum
    return numeratorSum / denominator

def findSSE( df: pd.DataFrame):
    mse=findMSE(df)
    return mse*(df.size - df.shape[1])

def findMST (df: pd.DataFrame):
    return findSST(df) / (df.shape[1] - 1)

def findF (df: pd.DataFrame):
    return findMST(df) / findMSE(df)

def findGrandMean(df: pd.DataFrame):
    sum = 0
    for columnName in df.columns:
        for element in df[columnName]:
            sum = element + sum
    return sum / df.size

def generateANOVAFrame (df: pd.DataFrame):
    treaments = df.columns.size - 1
    total = df.size - 1
    error = total - treaments 
    sst = round(findSST(df),3)
    sse = round(findSSE(df),3)
    tss = round(findTSS(df),3)
    mst = round(findMST(df),3)
    mse = round(findMSE(df),3)
    f_value = round(findF(df),3)
    p_value = round(f_oneway(*[df[column] for column in df.columns]).pvalue, 5)

    return pd.DataFrame({
        'source' : ['Treatments' , 'Error' , 'Total'],
        'df' : [treaments, error, total],
        'Sum of Squares' : [sst, sse, tss ],
        'Mean Squares' : [mst, mse , None],
        'F' : [f_value, None, None],
        'P-value' : [p_value,None, None]
    }).set_index('source')

def makeDetermination(df: pd.DataFrame, alpha = 0.05 ):
    p_value = round(f_oneway(*[df[column] for column in df.columns]).pvalue, 5)
    if p_value < alpha:
        return (True, str(alpha) + " > " + str(p_value) +  ": Reject Null Hypothesis at α=" + str(alpha))
    return (False, str(alpha) + " < " + str(p_value) +  ": Fail to Reject Null Hypothesis at α=" + str(alpha))

In [392]:
df1216 = pd.DataFrame({
    "strategy1" : [3,6,7,4,6,7,10,6,15,8,9,16],
    "strategy2" : [2,5,5,3,7,8,6,4,10,6,9,12],
    "strategy3" : [4,2,5,6,6,7,9,8,14,8,7,16]
})
print(findMSE(df1216))


13.287878787878784


***12.1 Problem 20***

In [393]:
df = pd.DataFrame({
    'day' : ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
    'firstShift' : [140,181,127,172,161,152,173],
    'secondShift' : [168,224,162,182,219,171,217],
    'thirdShift' : [77,123,77,101,147,145,111]
}).set_index('day')

In [394]:
sum = 0
for columnName in df.columns:
    sum = df[columnName].mean() + sum
print(str(sum/df.shape[1]))

153.80952380952382


In [395]:
print(df.shape[0])

7


In [396]:
print(df.shape[1])

3


In [397]:
df.size

21

***12.3 Problem 10***

In [398]:
df = pd.DataFrame({
    'minivanA': [150,152,151,149,153],
    'minivanB': [153,150,156,151,155],
    'minivanC' : [155,150,157,158,155],
    'minivanD': [167,164,169,162,173]
})

print(generateANOVAFrame(df))
print(makeDetermination(df, 0.10)[1])


            df  Sum of Squares  Mean Squares       F  P-value
source                                                       
Treatments   3           775.0       258.333  27.928      0.0
Error       16           148.0         9.250     NaN      NaN
Total       19           923.0           NaN     NaN      NaN
0.1 > 0.0: Reject Null Hypothesis at α=0.1
